Кирилл Лалаянц, R33352

# Лабораторная работа No4
## Астатизмы

Импорт необходимых для работы библиотек. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import control 
import sympy
import os
import math

SAVE_PATH = 'tex-report/figs/'
os.makedirs(SAVE_PATH, exist_ok=True)

sympy.init_printing()
p = sympy.Symbol("p")
s = sympy.Symbol("s")
t = sympy.Symbol("t")

def get_t(end_t = 10, dt=0.001, start_t = 0):
    return np.linspace(start_t, end_t, int(end_t / dt))

1. Задание

In [ ]:
def task1_tf(m1, m2):
    poly = sympy.simplify((p - m1) * (p - m2))
    print(poly.expand())
    coeffs = sympy.Poly(poly, p).all_coeffs()
    ss = control.tf2ss(control.tf(1, np.array(coeffs, dtype=np.float64)))
    ss_reachable = control.canonical_form(ss, form="reachable")[0]
    return ss_reachable

def task1_simulate(TF, ts, dt, g, u, closed_loop=True):
    gs = g(ts)
    ys = [1]
    dys = [1]
    es = [gs[0] - ys[0]]
    des = [0]
    us = []
    for indx, t in enumerate(ts):
        if indx < 1:
            continue
        # us.append(k0 * es[-1] + k1 * des[-1])
        us.append(u(es[-1], des[-1]))
        output = control.forced_response(TF, T=[t, t + dt], U=us[-1], X0=[dys[-1], ys[-1]])
        
        ys.append(output.outputs[-1])
        dys.append((ys[-1] - ys[-2]) / dt)
        if closed_loop:
            es.append(gs[indx] - ys[-1])
            des.append((es[-1] - es[-2]) / dt)
    ys = np.array(ys)
    us = np.array(us)
    gs = np.array(gs)
    dys = np.array(dys)
    des = np.array(des)
    es = np.array(es)
    return (ys, es, gs, us, dys, des)
  

In [ ]:
dt = 0.001
ts = get_t(10, dt=dt)
TF = task1_tf(-1, 1)
k0, k1 = 400, 2
g = lambda t: -1 + 0 * t 
u = lambda e, de: k0 * e + k1 * de

ys, es, gs, us, dys, des = task1_simulate(TF, ts, dt, g, u, closed_loop=False)
plt.plot(ts[:len(ys)], ys, label = 'y')
plt.plot(ts[:len(gs)], gs, label = 'goal')
plt.plot(ts[:len(es)], es, label = 'error')
plt.legend()
plt.grid()